In [1]:
import re
import itertools

l, r = 0, 1

def union(lst1, lst2):
    final_list = list(set().union(lst1, lst2))
    return final_list

def loadModel(modelPath):
	file = open(modelPath).read()
	K = (file.split("Variables:\n")[0].replace("Terminals:\n","").replace("\n",""))
	V = (file.split("Variables:\n")[1].split("Productions:\n")[0].replace("Variables:\n","").replace("\n",""))
	P = (file.split("Productions:\n")[1])
	return cleanAlphabet(K), cleanAlphabet(V), cleanrule(P)
#Make rule easy to work with
def cleanrule(expression):
	result = []
	#remove spaces and explode on ";"
	rawRulse = expression.replace('\n',';').split(';')
	
	for rule in rawRulse:
		#Explode evry rule on "->" and make a couple
		lSide = rule.split(' -> ')[0].replace(' ','')
		rTerms = rule.split(' -> ')[1].split(' | ')
		for term in rTerms:
			result.append( (lSide, term.strip().split(' ')) )
	return result

def cleanAlphabet(expression):
	return expression.replace('  ',' ').strip().split(' ')

def seekAndDestroy(target, rules):
	trash, ereased = [],[]
	for rule in rules:
		if target in rule[r] and len(rule[r]) == 1:
			trash.append(rule[l])
		else:
			ereased.append(rule)
			
	return trash, ereased
 
def setupDict(rules, variables, terms):
	result = {}
	for rule in rules:
		#
		if rule[l] in variables and rule[r][0] in terms and len(rule[r]) == 1:
			result[rule[r][0]] = rule[l]
	return result


def rewrite(target, rule):
	result = []
	#get positions corresponding to the occurrences of target in rule r side
	#positions = [m.start() for m in re.finditer(target, rule[r])]
	positions = [i for i,x in enumerate(rule[r]) if x == target]
	#for all found targets in rule
	for i in range(len(positions)+1):
 		#for all combinations of all possible lenght phrases of targets
 		for element in list(itertools.combinations(positions, i)):
 			#Example: if positions is [1 4 6]
 			#now i've got: [] [1] [4] [6] [1 4] [1 6] [4 6] [1 4 6]
 			#erease position corresponding to the target in rule r side
 			tadan = [rule[r][i] for i in range(len(rule[r])) if i not in element]
 			if tadan != []:
 				result.append((rule[l], tadan))
	return result

def dict2Set(dictionary):
	result = []
	for key in dictionary:
		result.append( (dictionary[key], key) )
	return result

def pprintRules(rules):
	for rule in rules:
		for term in rules[rule]:
		    if(len(term)==2):
		        print(rule," -> ",term[0],term[1])
		    else:
		        print(rule+" -> "+term[0])
		

def prettyForm(rules):
	dictionary = {}
	for rule in rules:
		if rule[l] in dictionary:
			dictionary[rule[l]] += ' | '+' '.join(rule[r])
		else:
			dictionary[rule[l]] = ' '.join(rule[r])
	result = ""
	for key in dictionary:
		result += key+" -> "+dictionary[key]+"\n"
	return result

In [2]:

l, r = 0, 1

K, V, rules = [],[],[]
nonTerminalsJar = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S",\
                   "T", "U", "W", "X", "Y", "Z"]


def isUnitary(rule, nonTerminals):
	if rule[l] in nonTerminals and rule[r][0] in nonTerminals and len(rule[r]) == 1:
		return True
	return False

def isSimple(rule):
	if rule[l] in V and rule[r][0] in K and len(rule[r]) == 1:
		return True
	return False


for nonTerminal in V:
	if nonTerminal in nonTerminalsJar:
		nonTerminalsJar.remove(nonTerminal)

#Add S0->S rule––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––START
def START(rules, nonTerminals):
	nonTerminals.append('S0')
	return [('S0', [nonTerminals[0]])] + rules
#Remove rules containing both terms and nonTerminals, like A->Bc, replacing by A->BZ and Z->c–––––––––––TERM
def TERM(rules, nonTerminals):
	result = []
	#create a dictionari for all base rule, like A->a, in the form dic['a'] = 'A'
	termDict = setupDict(rules, nonTerminals, terms=K)
	for rule in rules:
		#check if the rule is simple
		if isSimple(rule):
			#in that case there is nothing to change
			result.append(rule)
		else:
			for term in K:
				for index, value in enumerate(rule[r]):
					if term == value and not term in termDict:
						#it's created a new rule vaiable->term and added to it 
						termDict[term] = nonTerminalsJar.pop()
						#nonTerminals set it's updated adding new variable
						V.append(termDict[term])
						result.append( (termDict[term], [term]) )
						
						rule[r][index] = termDict[term]
					elif term == value:
						rule[r][index] = termDict[term]
			result.append( (rule[l], rule[r]) )
			
	#merge created set and the introduced rules
	return result

#Eliminate non unitry rules––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––BIN
def NONUNIT(rules, nonTerminals):
	result = []
	for rule in rules:
		k = len(rule[r])
		if k <= 2:
			result.append( rule )
		else:
			nv = nonTerminalsJar.pop(0)
			nonTerminals.append(nv+'1')
			result.append( (rule[l], [rule[r][0]]+[nv+'1']) )
			i = 1
#TODO
			for i in range(1, k-2 ):
				v1, v2 = nv+str(i), nv+str(i+1)
				nonTerminals.append(v2)
				result.append( (v1, [rule[r][i], v2]) )
			result.append( (nv+str(k-2), rule[r][k-2:]) ) 
	return result
	

#Delete non terminal rules–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––DEL
def DEL(rules):
	newSet = []
	#seekAndDestroy throw back in:
	#        – outlaws all l side of rules such that r side is equal to the outlaw
	#        – rules the rules without outlaws 
	outlaws, rules = seekAndDestroy(target='e',rules=rules)
	#add new reformulation of old rules
	for outlaw in outlaws:
		#consider every rule: old + new resulting important when more than one outlaws are in the same prod.
		for rule in rules + [e for e in newSet if e not in rules]:
			#if outlaw is present in the r side of a rule
			if outlaw in rule[r]:
				#the rule is rewrited in all combination of it, rewriting "e" rather than outlaw
				#this cycle prevent to insert duplicate rules
				newSet = newSet + [e for e in  rewrite(outlaw, rule) if e not in newSet]

	#add unchanged rules and return
	
	rules = newSet + ([rules[i] for i in range(len(rules)) if rules[i] not in newSet])
	return rules

def unit_elemination(rules, nonTerminals):
	unitRules, result = [], []
	for rule in rules:
		if isUnitary(rule, nonTerminals):
			unitRules.append( (rule[l], rule[r][0]) )
		else:
			result.append(rule)
	for uni in unitRules:
		for rule in rules:
			if uni[r]==rule[l] and uni[l]!=rule[l]:
				result.append( (uni[l],rule[r]) )
	return result

def UNIT(rules, nonTerminals):
	i = 0
	result = unit_elemination(rules, nonTerminals)
	tmp = unit_elemination(result, nonTerminals)
	while result != tmp and i < 1000:
		result = unit_elemination(tmp, nonTerminals)
		tmp = unit_elemination(result, nonTerminals)
		i+=1
	return result


        
def cykParsing(w,Rules):
	n = len(w)
	# Initialize the table
	CYK = [[set() for j in range(n+1)] for i in range(n)]

	# Filling in the table
	for j in range(0, n):
		# Iterate over the rs
		for l, r in Rules.items():
			for r in r:
				
				# If a terminal is found
				if len(r) == 1 and \
				r[0] == w[j]:
					CYK[j][j+1].add(l)
		for i in range(j-1, -1, -1): 
			# Iterate over the range i+1 to j 
			for k in range(i+1, j+1):	 
				# Iterate over the rs
				for l, r in Rules.items():
					for r in r:
						
						# If a terminal is found
						
						if len(r) == 2:
						    if r[0] in CYK[i][k] and r[1] in CYK[k][j+1]:
							    CYK[i][j+1].add(l)

	# If word can be formed by s 
	# of given grammar
# 	for i in w:
# 	    print("{:30}".format(i),end=" ")
# 	print("\n",end="")
# 	for i in range(n):
# 	    for j in range(1,n+1):
# 	        print("{:30}".format(CYK[i][j].__str__()),end=" ")
# 	    print("\n",end="")
	if len(CYK[0][n]) != 0 and ('S' in CYK[0][n] or 'S0' in CYK[0][n]):
		print("Yes")
	else:
		print("No")
		
def setToDict(rules):
    R = {}
    for i in rules:
        if i[l] not in R:
            R[i[l]]=[i[r]]
        else:
            R[i[l]].append(i[r])
    return R
    


modelPath = 'cfg.txt'

K, V, rules = loadModel( modelPath )

#rules = START(rules, nonTerminals=V)
rules = TERM(rules, nonTerminals=V)
rules = NONUNIT(rules, nonTerminals=V)
rules = DEL(rules)
rules = UNIT(rules, nonTerminals=V)


open('out.txt', 'w').write(	prettyForm(rules) ) #here we write the CNF grammer to a file named out.txt
R= setToDict(rules)
#pprintRules(R) #to print CNF grammer.
cykParsing(input("Please enter a sentence :- ").lower().strip().split(),R)

Please enter a sentence :- can i get some water
No
